<a href="https://colab.research.google.com/github/nehaa28/llm/blob/main/URL_based_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!pip install requests beautifulsoup4 urllib3 faiss-cpu transformers sentence-transformers langchain accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 663.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succes

# Code for the Web Scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import os

# Set to keep track of visited URLs
visited_urls = set()

def scrape_website(url, domain):
    if url in visited_urls:
        return
    visited_urls.add(url)
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract text content
    text = soup.get_text(separator=' ', strip=True)

    # Save text to a .txt file
    save_text(url, text)

    # Extract image and video links
    media_links = extract_media_links(soup, url)

    # Print media links (you can choose to save them as needed)
    for media_type, links in media_links.items():
        print(f"\n{media_type.upper()} LINKS in {url}:")
        for link in links:
            print(link)

    # Find and process all internal links recursively
    for link in soup.find_all('a', href=True):
        href = link['href']
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        if parsed_href.netloc == domain:
            scrape_website(href, domain)

def save_text(url, text):
    parsed_url = urlparse(url)
    path = parsed_url.path.strip('/')
    if not path:
        path = 'home'
    filename = f"{parsed_url.netloc}_{path.replace('/', '_')}.txt"
    os.makedirs('scraped_texts', exist_ok=True)
    filepath = os.path.join('scraped_texts', filename)
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"Saved text from {url} to {filepath}")
    except Exception as e:
        print(f"Failed to save text from {url}: {e}")

def extract_media_links(soup, base_url):
    media_links = {'image': [], 'video': []}
    # Images
    for img in soup.find_all('img', src=True):
        img_url = urljoin(base_url, img['src'])
        media_links['image'].append(img_url)
    # Videos
    for video in soup.find_all('video', src=True):
        video_url = urljoin(base_url, video['src'])
        media_links['video'].append(video_url)
    for source in soup.find_all('source', src=True):
        media_url = urljoin(base_url, source['src'])
        media_links['video'].append(media_url)
    return media_links

if __name__ == "__main__":
    start_url = input("Enter the URL to scrape: ").strip()
    parsed_url = urlparse(start_url)
    domain = parsed_url.netloc
    scrape_website(start_url, domain)


Enter the URL to scrape: https://www.web-scraping.dev/
Saved text from https://www.web-scraping.dev/ to scraped_texts/www.web-scraping.dev_home.txt

IMAGE LINKS in https://www.web-scraping.dev/:

VIDEO LINKS in https://www.web-scraping.dev/:
Saved text from https://www.web-scraping.dev/docs to scraped_texts/www.web-scraping.dev_docs.txt

IMAGE LINKS in https://www.web-scraping.dev/docs:

VIDEO LINKS in https://www.web-scraping.dev/docs:
Saved text from https://www.web-scraping.dev/api/graphql to scraped_texts/www.web-scraping.dev_api_graphql.txt

IMAGE LINKS in https://www.web-scraping.dev/api/graphql:

VIDEO LINKS in https://www.web-scraping.dev/api/graphql:
Saved text from https://www.web-scraping.dev/products to scraped_texts/www.web-scraping.dev_products.txt

IMAGE LINKS in https://www.web-scraping.dev/products:
https://web-scraping.dev/assets/products/orange-chocolate-box-medium-1.webp
https://web-scraping.dev/assets/products/darkred-potion.webp
https://web-scraping.dev/assets/pro

/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


# RAG

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3.1-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype='float16'
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto'
)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B.
401 Client Error. (Request ID: Root=1-66e3bbaa-136d30ee2afd0552721ce976;d298f626-a58c-4d33-b3d4-10c0315ec546)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3.1-8B is restricted. You must have access to it and be authenticated to access it. Please log in.

# Scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import os

# Set to keep track of visited URLs
visited_urls = set()

def get_all_links(url, base_domain):
    try:
        # Make a GET request to fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return []

    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Initialize an empty list to store the links found on this page
    links = []

    # Find all <a> tags with href attributes in the page
    for a_tag in soup.find_all('a', href=True):
        link = a_tag['href']
        # Join the base URL with the href to handle relative URLs
        full_link = urljoin(url, link)

        # Check if the domain of the link matches the base domain
        link_domain = urlparse(full_link).netloc
        if link_domain == base_domain and full_link not in visited_urls:
            links.append(full_link)

    return links

def save_page_content(url, folder_path):
    try:
        # Make a GET request to fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return

    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the page text
    page_text = soup.get_text(separator=' ', strip=True)

    # Create a file name based on the URL's path
    parsed_url = urlparse(url)
    file_name = parsed_url.path.strip('/').replace('/', '_') or 'home'
    file_path = os.path.join(folder_path, f"{file_name}.txt")

    # Save the text content to a .txt file
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(page_text)
        print(f"Saved content from {url} to {file_path}")
    except Exception as e:
        print(f"Failed to save content from {url}: {e}")

def crawl_website(url):
    # Parse the base domain from the input URL
    base_domain = urlparse(url).netloc

    # Create a folder with the base domain name to store the content
    folder_path = os.path.join(os.getcwd(), base_domain)
    os.makedirs(folder_path, exist_ok=True)

    # Recursively search and collect links within the same domain
    def recursive_crawl(current_url):
        if current_url in visited_urls:
            return
        visited_urls.add(current_url)

        # Get all internal links from the current page
        links = get_all_links(current_url, base_domain)

        # Save the page content to a .txt file
        save_page_content(current_url, folder_path)

        for link in links:
            if link not in visited_urls:
                recursive_crawl(link)  # Recursively crawl the new link

    # Start crawling from the initial URL
    recursive_crawl(url)

if __name__ == "__main__":
    # Input the URL
    start_url = input("Enter the URL to crawl: ").strip()

    # Start the crawling process
    crawl_website(start_url)

    # Print all the visited links
    print(f"\nCrawling completed. Total links found: {len(visited_urls)}")
    for link in visited_urls:
        print(link)
